# Understanding Session

In [1]:
from IPython.display import Markdown
import lionagi as li

The `Session` object is at the core of lionagi.

A session is an interface to
- manages and logs various messages (system, user, assistant) in a conversation, 
- call API with rate_limit, and 
- enable effortless multi-round exchange between many participants. 

A Session object contains a `Conversation` object, which contains an `Message` object. 
- `Conversation`: create, manage, keep track of messages in a single conversation
- `Message`: The smallest unit of data with a role


**Note:**
currently Session can only call ChatCompletion, support for other OpenAI API endpoints are under developments. currently only supports one assistant in a conversation, multi assistant will be supported in future

## 1. Initiate and Followup

Let us start with a simple use case of the `Session` object, which is to have a multi-turn conversation with an intelligence provider, i.e. OpenAI API service. 

In [2]:
# create some messages
sys_comedian = "As a comedian, you are sarcastically funny"
instruct1 = "very short joke: a blue whale and a big shark meet at the bar and start dancing"
instruct2 = "continue the short joke: the blue whale was confused why it was dancing with a shark"

In [3]:
# create a session with the comedian and generate some jokes in a multi-turn conversation
# also input target output directory for saving the conversation logs
comedian = li.Session(sys_comedian, dir="data/logs/comedian/")

joke1 = await comedian.initiate(instruct1)
joke2 = await comedian.followup(instruct2)

# you can continue the conversation with more messages until out of context length
# joke3 = await comedian.followup(instruct3)
# joke4 = await comedian.followup(instruct4)

In [4]:
Markdown(joke1)

So a blue whale and a big shark meet at a bar, right? They hit the dance floor, and the whale says, "Let's show 'em how to really make waves!" The shark replies, "Sure, but no fin-flapping, I just ironed my skin!"

In [5]:
Markdown(joke2)

The blue whale leans in and whispers, "I'm a little confused, aren't we supposed to be natural enemies?" The shark grins, "Only when the cameras are rolling. Off-screen, I'm just a big fish who can't resist a good krill-and-chill!"

## 2. Using a second Session as Validation

In [6]:
# some messages from the critic
sys_critic = "you are a respected commentator, you are artistically logical"
instruct3 = "short comments, what do you think about the first joke?"
instruct4 = "provide a concise artistic critique on both jokes, and rate from 1-10"

In [7]:
# create a session with the critic and generate some comments in a multi-turn conversation
critic = li.Session(sys_critic, dir="data/logs/critic/")

# provide context for the critic
context = {
    "joke1": {"prompt": instruct1, "response": joke1},
    "joke2": {"prompt": instruct2, "response": joke2}
}

# you only need to provide same context once in a conversation
comment1 = await critic.initiate(instruct3, context=context)    
comment2 = await critic.followup(instruct4)

In [8]:
Markdown(comment1)

The interplay of the unexpected—predatory marine creatures socializing playfully in a human setting—makes for a whimsical and light-hearted jest, though the punchline might benefit from a sharper twist to elevate the humor.

In [9]:
Markdown(comment2)

Joke 1 utilizes the absurdity of marine giants engaging in human activities, blending oceanic wordplay with a hint of anthropomorphism. The humor is gentle, yet it lacks an incisive punchline that could elevate the amusement. Rating: 5/10.

Joke 2 builds on the first by adding a layer of meta-humor—acknowledging the typical roles expected of these creatures in nature documentaries. The shark's line delivers a cheeky nod to behind-the-scenes reality versus public persona. Rating: 6/10 for the more developed concept and clever twist.

## 3. Using the Validation to improve first session performance

In [10]:
# messages for comedian again
instruct5 = "your jokes were evaluated by a critic, does it make sense to you? why?"
instruct6 = "based on your reflection, write joke1 again"
instruct7 = "write joke2 again"

# provide context for the comedian
context2 = {"comments": critic.conversation.responses}

# run the instructions
reflect = await comedian.followup(instruct5, context=context2)
joke11 = await comedian.followup(instruct6)
joke22 = await comedian.followup(instruct7)

In [11]:
Markdown(reflect)

Oh, absolutely, the critique makes perfect sense. In the complex world of humor, it's all about the delivery and the punch. The critic's point of view suggests that while the scenarios were amusing and the characters engaging, the punchlines could have been sharpened for a more impactful laugh. It's like serving a shark a seal-shaped cake—it’s amusing but would be better if it had some real bite to it.

It's crucial to remember that comedy is subjective, and what tickles one person's funny bone might just swim right past another's. Ratings like 5/10 and 6/10? Those are numbers I can work with! I mean, at least they didn't rate the jokes "watered down," right? But hey, even if they did, I'd just wave it off. Comedy is an ocean, and I'm just here to keep swimming!

In [12]:
Markdown(joke11)

So a blue whale and a big shark belly up to the bar and start busting moves on the dance floor. The DJ drops the beat, and the whale says, "Watch out, I've been known to bring the house down with my moves!" The shark winks and replies, "Yeah, and I'm known for my killer moves, but tonight I'm just here to take a bite out of the dance floor!"

In [13]:
Markdown(joke22)

The blue whale, a little puzzled, leans in and says, "Isn't it weird for a whale and a shark to be dancing together?" The shark flashes a toothy grin and replies, "Only if we were in a nature documentary. Here, we're just two sea-lings looking to have a whale of a time without any fishy business!"

#### Save the logs

In [14]:
# create seperate log for each session, and save both messages and llm api logs to seperate csv files

comedian.messages_to_csv()
comedian.log_to_csv()

critic.messages_to_csv()
critic.log_to_csv()

11 logs saved to data/logs/comedian/_messages_2023-12-17T20_44_11_937182.csv
5 logs saved to data/logs/comedian/_llmlog_2023-12-17T20_44_11_937781.csv
5 logs saved to data/logs/critic/_messages_2023-12-17T20_44_11_938869.csv
2 logs saved to data/logs/critic/_llmlog_2023-12-17T20_44_11_939154.csv
